In [276]:
"""
Подсчет базовых статистик для каждой из колонок с сохранением результатов в отдельные листы
"""

'\nПодсчет базовых статистик для каждой из колонок с сохранением результатов в отдельные листы\n'

In [277]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time
from iteration_utilities import duplicates

In [278]:
name_file_data_groupby = 'data/Эксперты ВСР.xlsx'
path_to_end_folder_groupby = 'data'

In [279]:
df = pd.read_excel(name_file_data_groupby)

In [280]:
# Создаем шрифт которым будем выделять названия таблиц
font_name_table = Font(name='Arial Black', size=15, italic=True)
# Создаем файл excel
wb = openpyxl.Workbook()

Колонки в таблице Index(['ФИО', 'Пробелы',
       'Организация, город   Отметка о пролонгации свидетельства', 'Регион',
       '№ свидетельства', 'Компетенция', 'Дата выдачи', 'Срок действия (лет)',
       'Телефон', 'Имейл'],
      dtype='object')


In [281]:
# Проверяем наличие возможных дубликатов ,котороые могут получиться если обрезать по 30 символов
lst_length_column = [column[:30] for column in df.columns]
check_dupl_length = list(duplicates(lst_length_column))

In [282]:
# проверяем наличие объединенных ячеек
check_merge = [ column for column in df.columns if 'Unnamed' in column]
# если есть хоть один Unnamed или дубликат то просто заменяем названия колонок на Колонка №цифра
if check_merge or check_dupl_length:
    df.columns = [f'Колонка №{i}' for i in range(1,df.shape[1]+1)]

In [284]:
# Делаем названия колонок строковыми
df.columns = list(map(str,list(df.columns)))

In [285]:
# Добавляем столбец для облегчения подсчета по категориям
df['Итого'] = 1

In [286]:
# Создаем листы
for idx,name_column in enumerate(df.columns):
    # Делаем короткое название не более 30 символов
    wb.create_sheet(title=name_column[:30],index=idx)

In [287]:
for idx,name_column in enumerate(df.columns):
    group_df = df[name_column].describe().to_frame()
    if group_df.shape[0] == 8:
        # подсчитаем сумму
        all_sum = df[name_column].sum()
        dct_row = {name_column: all_sum}
        row = pd.DataFrame(data=dct_row, index=['Сумма'])
        # Добавим в датафрейм
        group_df = pd.concat([group_df, row], axis=0)

        # Обновим названия индексов
        group_df.index = ['Количество значений', 'Среднее', 'Стандартное отклонение', 'Минимальное значение',
                          '25%(Первый квартиль)', 'Медиана', '75%(Третий квартиль)', 'Максимальное значение',
                          'Сумма']

    elif group_df.shape[0] == 4:
        group_df.index = ['Количество значений', 'Количество уникальных значений', 'Самое частое значение',
                          'Количество повторений самого частого значения', ]
    for r in dataframe_to_rows(group_df, index=True, header=True):
        if len(r) != 1:
            wb[name_column[:30]].append(r)
    wb[name_column[:30]].column_dimensions['A'].width = 50
    
    


In [288]:
# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Удаляем лист 
del wb['Sheet']
del wb['Итого']
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_groupby}/Подсчет базовых статистик для всех колонок таблицы от {current_time}.xlsx')